In [1]:
!export CUDA_VISIBLE_DEVICES=5

In [8]:
import os
import cv2
import sys
import numpy as np
from PIL import Image
from pathlib import Path
from paddle.utils import try_import

parent_dir = Path().resolve().parent
# # 插入模块路径
sys.path.insert(0, str(parent_dir))

from paddleocr import PPStructure,save_structure_res, PaddleOCR, draw_structure_result
from paddleocr.ppstructure.recovery.recovery_to_doc import sorted_layout_boxes, convert_info_docx, convert_info_markdown



ocr_engine = PPStructure(recovery=True, structure_version='PP-StructureV2')

save_folder = './layout_res'
pdf_path = '../data/basic_image.pdf'
font_path = '../fonts/simfang.ttf' # PaddleOCR下提供字体包

[2024/06/06 17:08:05] ppocr DEBUG: Namespace(help='==SUPPRESS==', use_gpu=True, use_xpu=False, use_npu=False, use_mlu=False, ir_optim=True, use_tensorrt=False, min_subgraph_size=15, precision='fp32', gpu_mem=500, gpu_id=0, image_dir=None, page_num=0, det_algorithm='DB', det_model_dir='/home/liangzhu/.paddleocr/whl/det/ch/ch_PP-OCRv4_det_infer', det_limit_side_len=960, det_limit_type='max', det_box_type='quad', det_db_thresh=0.3, det_db_box_thresh=0.6, det_db_unclip_ratio=1.5, max_batch_size=10, use_dilation=False, det_db_score_mode='fast', det_east_score_thresh=0.8, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_sast_score_thresh=0.5, det_sast_nms_thresh=0.2, det_pse_thresh=0, det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_scale=1, scales=[8, 16, 32], alpha=1.0, beta=1.0, fourier_degree=5, rec_algorithm='SVTR_LCNet', rec_model_dir='/home/liangzhu/.paddleocr/whl/rec/ch/ch_PP-OCRv4_rec_infer', rec_image_inverse=True, rec_image_shape='3, 48, 320', rec_batch_num=6, max_text

In [9]:
# 从 PDF 中获取页面图像
def covert_pdf_to_img(pdf_path):
    fitz = try_import("fitz")
    imgs = []

    with fitz.open(pdf_path) as pdf:
        for pg in range(0, pdf.page_count):
            page = pdf[pg]
            pm = page.get_pixmap(alpha=False)
            mat = fitz.Matrix(2, 2) # 将 PDF 页面缩放两倍，为了提高图像的分辨率和清晰度。
            pm = page.get_pixmap(matrix=mat, alpha=False)

            # if width or height > 2000 pixels, don't enlarge the image
            if pm.width > 2000 or pm.height > 2000:
                pm = page.get_pixmap(matrix=fitz.Matrix(1, 1), alpha=False)

            # 将像素图转换为 PIL 图像对象。
            img = Image.frombytes("RGB", [pm.width, pm.height], pm.samples) 
            
            # 将 PIL 图像转换为 NumPy 数组，并从 RGB 格式转换为 BGR 格式，以便 OpenCV 使用。
            img = cv2.cvtColor(np.array(img), cv2.COLOR_RGB2BGR) 
            imgs.append(img)
            
    return imgs

In [10]:
imgs = covert_pdf_to_img(pdf_path)
print(len(imgs))

2


获取页面的长宽像素。

print(page.bound().height)

print(page.bound().width)

In [13]:
def get_layout(imgs, ocr_engine):
    # for index, img in enumerate(imgs):
        index = 0
        img = imgs[0]
        result = ocr_engine(img)
        save_structure_res(result, save_folder, os.path.basename(pdf_path).split('.')[0], index)
        print(result)
        h, w, _ = img.shape
        res = sorted_layout_boxes(result, w)
        return res

In [14]:
layout_res = get_layout(imgs, ocr_engine)
print(layout_res)

[2024/06/06 17:11:28] ppocr DEBUG: dt_boxes num : 64, elapsed : 0.052954912185668945
[2024/06/06 17:11:29] ppocr DEBUG: rec_res num  : 64, elapsed : 0.17341399192810059
[2024/06/06 17:11:29] ppocr DEBUG: dt_boxes num : 28, elapse : 0.0247955322265625
[2024/06/06 17:11:29] ppocr DEBUG: rec_res num  : 28, elapse : 0.07010698318481445
[{'type': 'text', 'bbox': [621, 644, 951, 808], 'img': array([[[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       [[255, 255, 255],
        [255, 255, 255],
        [244, 244, 244],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       [[255, 255, 255],
        [255, 255, 255],
        [236, 236, 236],
        ...,
        [254, 254, 254],
        [255, 255, 255],
        [255, 255, 255]],

       ...,

       [[255, 255, 255],
        [254, 254, 254],
        [ 83,  83,  83],
        ...,
        [255, 2

In [15]:
res = convert_info_markdown(layout_res, save_folder, os.path.basename(pdf_path).split('.')[0])
print(res)

1.1标题-结果就产生了这个《宣言》，《宣言》原稿在二月革命前几星期送到伦敦付。不管最近 25年来的情况发生了多大的变化。当时(1847年12月)，卷入无产阶级运动的地区是多么狭小，这从《宣言》最后一章《共产党人对各国各种反对党派的态度》中可以看得很清楚。
## 1.2标题二

### 1.2.1第一部分

《宣言》最初用德文出版，它用这种文字在德国、英国和美国至少印过十二种不同的版本。第一个英译本是由海伦·麦克法林女士翻译的，于1850年在伦敦《红色共和党人》杂志上发表，1871年至少又有三种不同的英译本
1.2.1第二部分今天，情况完全不同了！正是欧洲移民，使北美能够进行大规模的农业生产，这种农业生产的竞争震撼着欧洲大小土地所有制的根基。此外，这种移民还使美国能够以巨大的力量。

![Figure 1 - This is a cat.](./layout_res/basic_image/%5B715%2C%20482%2C%20869%2C%20599%5D_0.jpg)
Figure 1 - This is a cat.
同时，在各工业区，人数众多的无产阶级和神话般的资本积聚第一次发展起来了。在波兰人中间，共产党人支持那个把土地革命当做民族解放的条件的政党，即发动过1846 年克拉科夫起义的政党。
表1水平表格
<table><tr><td>类型</td><td>制造商</td><td>模型</td><td>最高速度</td><td>平均成本</td><td>安全评级</td><td>燃料类型</td></tr><tr><td>汽车</td><td>丰田</td><td>卡罗拉</td><td>180 km/h</td><td>$20,000</td><td>5星</td><td>汽油</td></tr><tr><td>电动自行车</td><td>特斯拉</td><td>Model X Bike</td><td>25 km/h</td><td>$800</td><td>-</td><td>电</td></tr><tr><td>高速列车</td><td>法国国家铁路公司</td><td>TGV Duplex</td><td>320 km/h</td><td>$100 million</td><td>5星</td><td>电</td></tr></table>



##### draw_structure_result

In [7]:
for index, img in enumerate(imgs):
    result = ocr_engine(img)
    # 保存每一页的结构化结果图片
    im_show = draw_structure_result(Image.fromarray(cv2.cvtColor(img, cv2.COLOR_BGR2RGB)), result, font_path=font_path)
    im_show = Image.fromarray(im_show)
    im_show.save(os.path.join(save_folder, f'result_page_{index + 1}.jpg'))

[2024/06/06 17:07:24] ppocr DEBUG: dt_boxes num : 64, elapsed : 0.058054447174072266


[2024/06/06 17:07:24] ppocr DEBUG: rec_res num  : 64, elapsed : 0.20382213592529297
[2024/06/06 17:07:25] ppocr DEBUG: dt_boxes num : 28, elapse : 0.025774478912353516
[2024/06/06 17:07:25] ppocr DEBUG: rec_res num  : 28, elapse : 0.07493400573730469
[2024/06/06 17:07:25] ppocr DEBUG: dt_boxes num : 11, elapsed : 0.03747105598449707
[2024/06/06 17:07:25] ppocr DEBUG: rec_res num  : 11, elapsed : 0.16710186004638672
